Here I will go through the steps of determining the temperature of a gas of trapped atoms from spectroscopic data. As an example I will use lithium. I will use the packages that one typically uses to do this kind of analysis (Numpy, Scipy, Matplotlib).

We are analyzing a cloud of neutral Lithium 7 atoms in a trap. The cloud has some finite temperature. To probe the atoms we are scanning over a transition, the 2S1/2 to 2p3/2 transition. This atomic transition, like all (most?), has a lorentzian profile. There are several ways to show why this is. One is to model the electron as a spring attached to the nucleus, the result is a lorentzian response. Another way is to start with a lifetime, and from the uncertainty principle show that the frequency is has some finite width (cannot know the frequency and lifetime 100%). Since the atoms also have a temperature their velocity follows a gaussian distribution (velocity, not speed). This causes some atoms to be moving towards and some away from the laser, thus doppler shifting towards or away from resonance depending on the laser frequency.

The combination of these two effects is the convolution of a gaussian and a lorentzian, which gives a voigt. For given spectroscopic data, to extract the temperature one must fit the data with a voigt and then use the optimal values to determine the temperature. That is what we will do here. We will also try to determine the density of the cloud.